# Neural Networks
- Vidhish Trivedi (IMT2021055)
- Barath S Narayan (IMT2021524)
- Vikas Kalyanapuram (IMT2021040)

## Importing The Libraries

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf

## Loading The Data, Feature Engineering

In [ ]:
# Load data
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

train_frequency = train_data['patient_id'].value_counts().to_dict()
test_frequency = test_data['patient_id'].value_counts().to_dict()
frequency = {}

for i in train_frequency:
    frequency[i] = 0
for i in test_frequency:
    frequency[i] = 0

for i in train_frequency:
    frequency[i] += train_frequency[i]
for i in test_frequency:
    frequency[i] += test_frequency[i]

train_data['frequency_pid'] = train_data['patient_id'].map(frequency)
test_data['frequency_pid'] = test_data['patient_id'].map(frequency)

## Preprocessing

In [ ]:
# Assuming 'patient_id' and 'enc_id' are not features for training
features = train_data.drop(['patient_id', 'enc_id', 'readmission_id'], axis=1)
labels = to_categorical(train_data['readmission_id'])  # One-hot encoding the labels

# Handling missing values and encoding categorical variables
numeric_features = features.select_dtypes(include=['int64', 'float64']).columns
categorical_features = features.select_dtypes(include=['object']).columns

# Create a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)])

# Splitting the training data for training and validation
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

# Apply the preprocessing
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
test_data_processed = preprocessor.transform(test_data.drop(['patient_id', 'enc_id'], axis=1))

## Building The Neural Network

In [4]:
# Building the neural network for multi-class classification
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


## Training The Model

In [5]:
# Training the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10


1781/1781 [==============================] - 9s 4ms/step - loss: 0.7940 - accuracy: 0.6566 - val_loss: 0.7364 - val_accuracy: 0.6939
Epoch 2/10
1781/1781 [==============================] - 8s 4ms/step - loss: 0.7251 - accuracy: 0.7013 - val_loss: 0.7190 - val_accuracy: 0.7104
Epoch 3/10
1781/1781 [==============================] - 8s 4ms/step - loss: 0.7053 - accuracy: 0.7134 - val_loss: 0.7101 - val_accuracy: 0.7122
Epoch 4/10
1781/1781 [==============================] - 9s 5ms/step - loss: 0.6940 - accuracy: 0.7191 - val_loss: 0.7043 - val_accuracy: 0.7143
Epoch 5/10
1781/1781 [==============================] - 9s 5ms/step - loss: 0.6840 - accuracy: 0.7217 - val_loss: 0.7052 - val_accuracy: 0.7143
Epoch 6/10
1781/1781 [==============================] - 9s 5ms/step - loss: 0.6769 - accuracy: 0.7248 - val_loss: 0.7007 - val_accuracy: 0.7157
Epoch 7/10
1781/1781 [==============================] - 9s 5ms/step - loss: 0.6735 - accuracy: 0.7245 - val_loss: 0.7126 - val_accurac

## Making Predictions

In [6]:
# Making predictions
predictions = model.predict(test_data_processed)
predicted_labels = np.argmax(predictions, axis=1)

output = pd.DataFrame({'enc_id': test_data['enc_id'], 'readmission_id': predicted_labels})
output.to_csv('./output.csv', index=False)


955/955 [==============================] - 2s 2ms/step
